# Tarefa 2 - MO432A

Alunos:
* Andreza - RA: 164213
* Gil - RA: 225323
* Yan - RA: 118982

## Ler o arquivo

Trata-se do arquivo Bias_correction_ucl.csv, devendo ser removida a coluna Next_Tmin, a coluna Date,e as linhas que tem valor faltante.

In [1]:
import sklearn
import pandas as pd
import numpy as np
import random

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, cross_validate
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import LinearSVR, SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

In [2]:
df_original = pd.read_csv('Bias_correction_ucl.csv')
df_original.head()

,station,Date,Present_Tmax,Present_Tmin,LDAPS_RHmin,LDAPS_RHmax,LDAPS_Tmax_lapse,LDAPS_Tmin_lapse,LDAPS_WS,LDAPS_LH,...,LDAPS_PPT2,LDAPS_PPT3,LDAPS_PPT4,lat,lon,DEM,Slope,Solar radiation,Next_Tmax,Next_Tmin
0,1.0,2013-06-30,28.7,21.4,58.255688,91.116364,28.074101,23.006936,6.818887,69.451805,...,0.0,0.0,0.0,37.6046,126.991,212.3350,2.7850,5992.895996,29.1,21.2
1,2.0,2013-06-30,31.9,21.6,52.263397,90.604721,29.850689,24.035009,5.691890,51.937448,...,0.0,0.0,0.0,37.6046,127.032,44.7624,0.5141,5869.312500,30.5,22.5
2,3.0,2013-06-30,31.6,23.3,48.690479,83.973587,30.091292,24.565633,6.138224,20.573050,...,0.0,0.0,0.0,37.5776,127.058,33.3068,0.2661,5863.555664,31.1,23.9
3,4.0,2013-06-30,32.0,23.4,58.239788,96.483688,29.704629,23.326177,5.650050,65.727144,...,0.0,0.0,0.0,37.6450,127.022,45.7160,2.5348,5856.964844,31.7,24.3
4,5.0,2013-06-30,31.4,21.9,56.174095,90.155128,29.113934,23.486480,5.735004,107.965535,...,0.0,0.0,0.0,37.5507,127.135,35.0380,0.5055,5859.552246,31.2,22.5


In [3]:
df = df_original.drop(['Date', 'Next_Tmin'], axis=1)
print("Número de linhas: {}\nNúmero de colunas: {}".format(df.shape[0], df.shape[1]))
df.head()

Número de linhas: 7752
Número de colunas: 23


,station,Present_Tmax,Present_Tmin,LDAPS_RHmin,LDAPS_RHmax,LDAPS_Tmax_lapse,LDAPS_Tmin_lapse,LDAPS_WS,LDAPS_LH,LDAPS_CC1,...,LDAPS_PPT1,LDAPS_PPT2,LDAPS_PPT3,LDAPS_PPT4,lat,lon,DEM,Slope,Solar radiation,Next_Tmax
0,1.0,28.7,21.4,58.255688,91.116364,28.074101,23.006936,6.818887,69.451805,0.233947,...,0.0,0.0,0.0,0.0,37.6046,126.991,212.3350,2.7850,5992.895996,29.1
1,2.0,31.9,21.6,52.263397,90.604721,29.850689,24.035009,5.691890,51.937448,0.225508,...,0.0,0.0,0.0,0.0,37.6046,127.032,44.7624,0.5141,5869.312500,30.5
2,3.0,31.6,23.3,48.690479,83.973587,30.091292,24.565633,6.138224,20.573050,0.209344,...,0.0,0.0,0.0,0.0,37.5776,127.058,33.3068,0.2661,5863.555664,31.1
3,4.0,32.0,23.4,58.239788,96.483688,29.704629,23.326177,5.650050,65.727144,0.216372,...,0.0,0.0,0.0,0.0,37.6450,127.022,45.7160,2.5348,5856.964844,31.7
4,5.0,31.4,21.9,56.174095,90.155128,29.113934,23.486480,5.735004,107.965535,0.151407,...,0.0,0.0,0.0,0.0,37.5507,127.135,35.0380,0.5055,5859.552246,31.2


In [4]:
df.dropna(inplace = True)
print("Número de linhas: {}\nNúmero de colunas: {}".format(df.shape[0], df.shape[1]))
df.head()

Número de linhas: 7588
Número de colunas: 23


,station,Present_Tmax,Present_Tmin,LDAPS_RHmin,LDAPS_RHmax,LDAPS_Tmax_lapse,LDAPS_Tmin_lapse,LDAPS_WS,LDAPS_LH,LDAPS_CC1,...,LDAPS_PPT1,LDAPS_PPT2,LDAPS_PPT3,LDAPS_PPT4,lat,lon,DEM,Slope,Solar radiation,Next_Tmax
0,1.0,28.7,21.4,58.255688,91.116364,28.074101,23.006936,6.818887,69.451805,0.233947,...,0.0,0.0,0.0,0.0,37.6046,126.991,212.3350,2.7850,5992.895996,29.1
1,2.0,31.9,21.6,52.263397,90.604721,29.850689,24.035009,5.691890,51.937448,0.225508,...,0.0,0.0,0.0,0.0,37.6046,127.032,44.7624,0.5141,5869.312500,30.5
2,3.0,31.6,23.3,48.690479,83.973587,30.091292,24.565633,6.138224,20.573050,0.209344,...,0.0,0.0,0.0,0.0,37.5776,127.058,33.3068,0.2661,5863.555664,31.1
3,4.0,32.0,23.4,58.239788,96.483688,29.704629,23.326177,5.650050,65.727144,0.216372,...,0.0,0.0,0.0,0.0,37.6450,127.022,45.7160,2.5348,5856.964844,31.7
4,5.0,31.4,21.9,56.174095,90.155128,29.113934,23.486480,5.735004,107.965535,0.151407,...,0.0,0.0,0.0,0.0,37.5507,127.135,35.0380,0.5055,5859.552246,31.2


In [5]:
x = df.drop(['Next_Tmax'], axis=1)
x = StandardScaler().fit_transform(x)
x = pd.DataFrame(x)
print("Número de linhas: {}\nNúmero de colunas: {}".format(x.shape[0], x.shape[1]))
x.head()

Número de linhas: 7588
Número de colunas: 22


,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,-1.664607,-0.353318,-0.748029,0.104660,0.382768,-0.525269,-0.215525,-0.126423,0.206603,-0.513123,...,-0.660441,-0.305589,-0.275777,-0.239969,-0.224971,1.186076,-0.005302,2.769091,1.111162,1.510565
1,-1.526052,0.725138,-0.664721,-0.305052,0.311697,0.078334,0.223368,-0.644133,-0.313359,-0.545304,...,-0.673074,-0.305589,-0.275777,-0.239969,-0.224971,1.186076,0.512280,-0.315828,-0.543220,1.222997
2,-1.387498,0.624033,0.043400,-0.549344,-0.609425,0.160080,0.449896,-0.439100,-1.244497,-0.606944,...,-0.616249,-0.305589,-0.275777,-0.239969,-0.224971,0.650626,0.840503,-0.526719,-0.723891,1.209602
3,-1.248943,0.758840,0.085054,0.103573,1.128335,0.028710,-0.079238,-0.663353,0.096026,-0.580143,...,-0.647336,-0.305589,-0.275777,-0.239969,-0.224971,1.987268,0.386040,-0.298272,0.928888,1.194265
4,-1.110389,0.556630,-0.539758,-0.037665,0.249244,-0.171981,-0.010803,-0.624327,1.349989,-0.827872,...,-0.506152,-0.305589,-0.275777,-0.239969,-0.224971,0.117159,1.812547,-0.494848,-0.549485,1.200286


In [6]:
y = df.get(['Next_Tmax'])
print("Número de linhas: {}\nNúmero de colunas: {}".format(y.shape[0], y.shape[1]))
y.head()

Número de linhas: 7588
Número de colunas: 1


,Next_Tmax
0,29.1
1,30.5
2,31.1
3,31.7
4,31.2


## Cross validation, medida de erro e busca de hiperparametros

* Usar 5-fold cross validation
* Usar RMSE como medida de erro
* Criar distribuição uniforme para a busca aleatório de hiperparametro, dnetro do intervalo especificado em cada problema/regressor

## Para cada um dos regressores abaixo

* Reportar o RMSE da melhor combinação de hiperparametros e o valor dos hiperparametros encontrados
* Reportar o RMSE do uso dos valores default do SKLearn para os hiperparametros que buscamos

### Linear

In [7]:
cross_val = cross_validate(
    LinearRegression(),
    x,
    y,
    scoring=('neg_root_mean_squared_error')
)
rmse_medio =  np.sqrt(np.mean(np.absolute(cross_val['test_score'])))
print("RMSE = {}".format(rmse_medio))

RMSE = 1.2560040654482534


### Regularização L2

In [8]:
alpha_list = 10**(np.random.uniform(-3, 3, 10))
final_rmse = np.inf
for alpha_value in alpha_list:
    cross_val = cross_validate(
        Ridge(alpha_value), 
        x, y,
        scoring=('neg_root_mean_squared_error')
    )
    rmse_medio = np.sqrt(np.mean(np.absolute(cross_val['test_score'])))
    if rmse_medio < final_rmse:
        final_rmse = rmse_medio
        final_alpha = alpha_value
print("Resultado da busca por hiperparâmetros:")
print("Melhor RMSE = {}".format(final_rmse))
print("Melhor alpha = {}".format(final_alpha))

Resultado da busca por hiperparâmetros:
Melhor RMSE = 1.2538136417832648
Melhor alpha = 260.8932240691377


In [9]:
cross_val = cross_validate(
            Ridge(), 
            x, y,
            scoring=('neg_root_mean_squared_error'))
rmse_medio = np.sqrt(np.mean(np.absolute(cross_val['test_score'])))
print("Resultado usando valores default:")
print("RMSE = {}".format(rmse_medio))

Resultado usando valores default:
RMSE = 1.2559766342954395


### Regularização L1

In [10]:
alpha_list = 10**(np.random.uniform(-3, 3, 10))
final_rmse = np.inf
for alpha_value in alpha_list:
    cross_val = cross_validate(
        Lasso(alpha_value),
        x,
        y,
        scoring=('neg_root_mean_squared_error')
    )
    rmse_medio = np.sqrt(np.mean(np.absolute(cross_val['test_score'])))
    if rmse_medio < final_rmse:
        final_rmse = rmse_medio
        final_alpha = alpha_value
print("Melhor RMSE = {}".format(final_rmse))
print("Valor do alpha = {}".format(final_alpha))

Melhor RMSE = 1.2520955810371297
Valor do alpha = 0.033787779100535036


In [11]:
cross_val = cross_validate(
            Lasso(), 
            x, y,
            scoring=('neg_root_mean_squared_error'))
rmse_medio = np.sqrt(np.mean(np.absolute(cross_val['test_score'])))
print("Resultado usando valores default:")
print("RMSE = {}".format(rmse_medio))

Resultado usando valores default:
RMSE = 1.4227986510522177


### SVM Linear

In [12]:
epsilon_list = [0.1, 0.3]
C_list = 2**(np.random.uniform(-5, 15, 10))
final_rmse = np.inf
for C_value in C_list:
    epsilon_value = random.choice(epsilon_list)
    cross_val = cross_validate(
        LinearSVR(epsilon = epsilon_value, C = C_value, max_iter = 3000),
        x,
        np.ravel(y),
        scoring=('neg_root_mean_squared_error')
    )
    rmse_medio = np.sqrt(np.mean(np.absolute(cross_val['test_score'])))
    if rmse_medio < final_rmse:
        final_rmse = rmse_medio
        final_epsilon = epsilon_value
        final_C = C_value
print("Melhor RMSE = {}".format(final_rmse))
print("Valor do epsilon = {}".format(final_epsilon))
print("Valor do C = {}".format(final_C))

/home/andrezasa/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/home/andrezasa/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/home/andrezasa/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/home/andrezasa/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/home/andrezasa/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to con

/home/andrezasa/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


Melhor RMSE = 1.2486185826470806
Valor do epsilon = 0.1
Valor do C = 0.126679923000806


/home/andrezasa/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [14]:
cross_val = cross_validate(
        LinearSVR(),
        x,
        np.ravel(y),
        scoring=('neg_root_mean_squared_error')
    )
rmse_medio = np.sqrt(np.mean(np.absolute(cross_val['test_score'])))
print("Resultado usando valores default:")
print("RMSE = {}".format(rmse_medio))

/home/andrezasa/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/home/andrezasa/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/home/andrezasa/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/home/andrezasa/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


Resultado usando valores default:
RMSE = 1.250411881995019


/home/andrezasa/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


### SVM com kernel RBF

In [15]:
epsilon_list = [0.1, 0.3]
C_list = 2**(np.random.uniform(-5, 15, 10))
gamma_list = 2**(np.random.uniform(-9, 3, 10))
final_rmse = np.inf
                
for C_value in C_list:
    epsilon_value = random.choice(epsilon_list)
    gamma_value = random.choice(gamma_list)
    cross_val = cross_validate(
        SVR(gamma = gamma_value,
            C = C_value,
            epsilon = epsilon_value,
            kernel='rbf'),
        x,
        np.ravel(y),
        scoring=('neg_root_mean_squared_error')
    )
    rmse_medio = np.sqrt(np.mean(np.absolute(cross_val['test_score'])))
    if rmse_medio < final_rmse:
        final_rmse = rmse_medio
        final_epsilon = epsilon_value
        final_C = C_value
        final_gamma = gamma_value

print("Melhor RMSE = {}".format(final_rmse))
print("Valor do epsilon = {}".format(final_epsilon))
print("Valor do C = {}".format(final_C))
print("Valor do gamma = {}".format(final_gamma))

Melhor RMSE = 1.253754334249183
Valor do epsilon = 0.3
Valor do C = 0.6684217705929327
Valor do gamma = 0.007474445215789686


In [16]:
cross_val = cross_validate(
        SVR(),
        x,
        np.ravel(y),
        scoring=('neg_root_mean_squared_error')
    )
rmse_medio = np.sqrt(np.mean(np.absolute(cross_val['test_score'])))
print("Resultado usando valores default:")
print("RMSE = {}".format(rmse_medio))

Resultado usando valores default:
RMSE = 1.299228674531788


### KNN

In [17]:
K_list = random.sample(range(1, 1001), 10)
final_rmse = np.inf
for K_value in K_list:
    cross_val = cross_validate(
        KNeighborsRegressor(K_value),
        x,
        y,
        scoring=('neg_root_mean_squared_error')
    )
    rmse_medio = np.sqrt(np.mean(np.absolute(cross_val['test_score'])))
    if rmse_medio < final_rmse:
        final_rmse = rmse_medio
        final_K = K_value
print("Melhor RMSE = {}".format(final_rmse))
print("Valor do K = {}".format(final_K))

Melhor RMSE = 1.3757670659831496
Valor do K = 157


In [18]:
cross_val = cross_validate(
        KNeighborsRegressor(),
        x,
        y,
        scoring=('neg_root_mean_squared_error')
    )
rmse_medio = np.sqrt(np.mean(np.absolute(cross_val['test_score'])))
print("Resultado usando valores default:")
print("RMSE = {}".format(rmse_medio))

Resultado usando valores default:
RMSE = 1.3881824886378122


### MLP

In [27]:
final_rmse = np.inf
for Hlayers_value in range(5, 21, 3):
    cross_val = cross_validate(
        MLPRegressor(Hlayers_value),
        x,
        np.ravel(y),
        scoring=('neg_root_mean_squared_error')
    )
    rmse_medio = np.sqrt(np.mean(np.absolute(cross_val['test_score'])))
    if rmse_medio < final_rmse:
        final_rmse = rmse_medio
        final_Hlayers = Hlayers_value
print("Melhor RMSE = {}".format(final_rmse))
print("Camadas escondidas = {}".format(final_Hlayers))

/home/andrezasa/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/andrezasa/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/andrezasa/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/andrezasa/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/andrezasa/.local/lib/python3.8/sit

Melhor RMSE = 1.3342414445487016
Camadas escondidas = 8


/home/andrezasa/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(


In [20]:
cross_val = cross_validate(
        MLPRegressor(),
        x,
        np.ravel(y),
        scoring=('neg_root_mean_squared_error')
)
rmse_medio = np.sqrt(np.mean(np.absolute(cross_val['test_score'])))
print("Resultado usando valores default:")
print("RMSE = {}".format(rmse_medio))

/home/andrezasa/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/andrezasa/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/andrezasa/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/andrezasa/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Resultado usando valores default:
RMSE = 1.4146059679908776


/home/andrezasa/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


### Árvore de decisão

In [21]:
ccp_alpha_list = np.random.uniform(0, 0.04, 10)
final_rmse = np.inf
for ccp_alpha_value in ccp_alpha_list:
    cross_val = cross_validate(
        DecisionTreeRegressor(ccp_alpha = ccp_alpha_value),
        x,
        y,
        scoring=('neg_root_mean_squared_error')
    )
    rmse_medio = np.sqrt(np.mean(np.absolute(cross_val['test_score'])))
    if rmse_medio < final_rmse:
        final_rmse = rmse_medio
        final_ccp_alpha = ccp_alpha_value
print("Melhor RMSE = {}".format(final_rmse))
print("Valor do ccp_alpha = {}".format(final_ccp_alpha))

Melhor RMSE = 1.3588060137522582
Valor do ccp_alpha = 0.03723309384171922


In [22]:
cross_val = cross_validate(
        DecisionTreeRegressor(),
        x,
        y,
        scoring=('neg_root_mean_squared_error')
    )
print("Resultado usando valores default:")
print("RMSE = {}".format(rmse_medio))

Resultado usando valores default:
RMSE = 1.3977159845546736


### Random Forest

In [23]:
n_estimators_list = [10, 100, 1000]
max_features_list = [5, 10, 22]
final_rmse = np.inf
for n_estimators_value in n_estimators_list:
    for max_features_value in max_features_list:
        cross_val = cross_validate(
            RandomForestRegressor(n_estimators = n_estimators_value, max_features = max_features_value),
            x,
            np.ravel(y),
            scoring=('neg_root_mean_squared_error')
        )
        rmse_medio = np.sqrt(np.mean(np.absolute(cross_val['test_score'])))
        if rmse_medio < final_rmse:
            final_rmse = rmse_medio
            final_n_estimators = n_estimators_value
            final_max_features = max_features_value
print("Melhor RMSE = {}".format(final_rmse))
print("Valor de estimadores = {}".format(final_n_estimators))
print("Valor máximo de atributos = {}".format(final_max_features))

Melhor RMSE = 1.2748278545285967
Valor de estimadores = 1000
Valor máximo de atributos = 10


In [24]:
cross_val = cross_validate(
    RandomForestRegressor(),
    x,
    np.ravel(y),
    scoring=('neg_root_mean_squared_error')
)
rmse_medio = np.sqrt(np.mean(np.absolute(cross_val['test_score'])))
print("Resultado usando valores default:")
print("RMSE = {}".format(rmse_medio))

Resultado usando valores default:
RMSE = 1.2855642897501085


### GBM

In [25]:
n_estimators_list = random.sample(range(5, 101), 10)
learning_rate_list = np.random.uniform(0.01, 0.3, 10)
max_depth_list = [2, 3]
final_rmse = np.inf

for n_estimators_value in n_estimators_list:
    learning_rate_value = random.choice(learning_rate_list)
    max_depth_value = random.choice(max_depth_list)
    cross_val = cross_validate(
        GradientBoostingRegressor(
            learning_rate = learning_rate_value,
            n_estimators = n_estimators_value,
            max_features = max_features_value
        ),
        x,
        np.ravel(y),
        scoring=('neg_root_mean_squared_error')
    )
    rmse_medio = np.sqrt(np.mean(np.absolute(cross_val['test_score'])))
    if rmse_medio < final_rmse:
        final_rmse = rmse_medio
        final_n_estimators = n_estimators_value
        final_learning_rate = learning_rate_value
        final_max_depth = max_depth_value

print("Melhor RMSE = {}".format(final_rmse))
print("Valor de estimadores = {}".format(final_n_estimators))
print("Valor da taxa de aprendizado = {}".format(final_learning_rate))
print("Valor da profundidade máxima = {}".format(final_max_depth))

Melhor RMSE = 1.2678967447966512
Valor de estimadores = 50
Valor da taxa de aprendizado = 0.09988174079350304
Valor da profundidade máxima = 2


In [26]:
cross_val = cross_validate(
    GradientBoostingRegressor(),
    x,
    np.ravel(y),
    scoring=('neg_root_mean_squared_error')
)
rmse_medio = np.sqrt(np.mean(np.absolute(cross_val['test_score'])))
rmse_medio = np.sqrt(np.mean(np.absolute(cross_val['test_score'])))
print("Resultado usando valores default:")
print("RMSE = {}".format(rmse_medio))

Resultado usando valores default:
RMSE = 1.263458065868345
